<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/2026-1/notebooks/0b_autodiff_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diferenciación automática
La [diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation) es un método para evaluar derivadas de funciones representadas como programas [[Automatic Differentiation in Machine Learning: a Survey, Baydin et. al, 2018](https://arxiv.org/abs/1502.05767)].

![Diferenciación automática](https://gowrishankar.info/blog/automatic-differentiation-using-gradient-tapes/auto_diff.png)

Fuente: [Automatic Differentiation in Machine Learning: a Survey, Baydin et. al, 2018](https://arxiv.org/abs/1502.05767).

In [1]:
import numpy as np
import torch as th
from torch import nn

th.manual_seed(42)
np.random.seed(42)

## Clase `Parameter`
La clase [`Parameter`](https://www.tensorflow.org/api_docs/python/tf/Variable) del módulo `nn` de PyTorch define una subclase de `Tensor` que se emplea comúnmente para representar los parámetros que modifican los algoritmos de aprendizaje para generar un modelo. Las instancias de `Parameter` que se definen dentro de una subclase de `Module` del módulo de `nn` se agregan a su lista de parámetros a optimizar.

El constructor de la clase `Parameter` recibe un tensor como argumento con el que se crea la instancia.


In [2]:
print(nn.parameter.Parameter(th.zeros((10,5))))
print(nn.parameter.Parameter(th.rand((10,5))))
print(nn.parameter.Parameter(th.ones((5,5))))
print(nn.parameter.Parameter(th.tensor([[1.0, 2.0], [3.0, 4.0]])))

Parameter containing:
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904],
        [0.6009, 0.2566, 0.7936, 0.9408, 0.1332],
        [0.9346, 0.5936, 0.8694, 0.5677, 0.7411],
        [0.4294, 0.8854, 0.5739, 0.2666, 0.6274],
        [0.2696, 0.4414, 0.2969, 0.8317, 0.1053],
        [0.2695, 0.3588, 0.1994, 0.5472, 0.0062],
        [0.9516, 0.0753, 0.8860, 0.5832, 0.3376],
        [0.8090, 0.5779, 0.9040, 0.5547, 0.3423],
        [0.6343, 0.3644, 0.7104, 0.9464, 0.7890],
        [0.2814, 0.7886, 0.5895, 0.7539, 0.1952]], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
  

Podemos realizar cualquier operación tensorial con una instancia de `Parameter`, ya sea con otras instancias de `Parameter` o `Tensor`. El resultado de la operación es una instancia de `Tensor`.

In [3]:
param = nn.parameter.Parameter(th.zeros(10, 5))

print(param.T)
print(param + th.ones_like(param))
print(param * th.zeros_like(param))
print(param @ th.rand((5, 10)))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<PermuteBackward0>)
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], grad_fn=<AddBackward0>)
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], grad_fn=<MulBackward0>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 

## Diferenciación automática en PyTorch
PyTorch puede diferenciar automáticamente secuencias de operaciones con instancias de `Tensor` o `Parameter`. Para ello se mantiene una gráfica de cómputo, la cual se va generando de manera dinámica conforme se ejecutan operaciones con instancias que tienen la propiedad `requires_grad` en verdadero (solo ten cuidado con las operaciones _in-place_). Por defecto, todas las instancias de `Parameter` tienen esta propiedad en verdadero y las instancias de `Tensor` en falso.

In [4]:
print(param.requires_grad)

True


In [5]:
ten = th.zeros((10, 5))

print(ten.requires_grad)

False


Es posible cambiar el valor de esta propiedad en una instancia usando el método `requires_grad_` (_in-place_).

In [6]:
param.requires_grad_(False)
ten.requires_grad_(True)
print(ten.requires_grad, param.requires_grad)

True False


In [7]:
param.T, ten.T

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<PermuteBackward0>))

In [8]:
param.requires_grad_(True)

Parameter containing:
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], requires_grad=True)

Para modificar el contenido de una instancia de `Parameter` es necesario especificar que no se registre la operación usando el ámbito `no_grad`.

In [9]:
with th.no_grad():
  param[0, 0] = 1

In [10]:
param

Parameter containing:
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], requires_grad=True)

En general, existen métodos para instancias tanto de `Parameter` como de `Tensor` que modifican el contenido _in-place_. Los nombres de estos métodos usualmente terminan con un guión bajo. Tal es el caso de `add_` y `mul_`, que suman y multiplican un tensor. Debido a que estas operaciones no deben registrarse cuando las instancias de `Parameter` o `Tensor` tienen `requires_grad = True`, las ejecutamos dentro del ámbito `no_grad`.

In [11]:
with th.no_grad():
  param.add_(th.ones_like(param))
  param.mul_(-th.ones_like(param))
  param.sub_(2 * th.ones_like(param))
print(param)

Parameter containing:
tensor([[-4., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.],
        [-3., -3., -3., -3., -3.]], requires_grad=True)


Para obtener el gradiente de una [función escalar](https://en.wiktionary.org/wiki/scalar_function) respecto a un tensor que está siendo contemplado se debe llamar al método `backward`.

Por ejemplo, considera la siguiente función:

$$
f(x, y) = 2x^3 + 3y^2 + c
$$

![](https://camo.githubusercontent.com/32e5a6b4cf8ad981ba5b19c9a5fab02fdc14d409237f774034d23536a731ae5c/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f626572656d6c2f6961702f6d61737465722f6669672f6175746f646966665f6578616d706c652e737667)

Ejemplo y figura de Berenice Montalvo-Lezama y Ricardo Montalvo-Lezama (tomado de [https://github.com/gibranfp/CursoAprendizajeProfundo/blob/2022-1/notebooks/1c_pytorch.ipynb](https://github.com/gibranfp/CursoAprendizajeProfundo/blob/2022-1/notebooks/1c_pytorch.ipynb)).

Evaluando esta función en $x = 2$, $y = 3$ y $c = 1.0$, tenemos

$$
f(2, 3) = 2\cdot (2)^3 + 3 \cdot (3)^2 + 1 = 44
$$

Las derivadas partiales evaluadas en estos puntos estarían dadas por
$$
\begin{align}
\frac{\partial f}{\partial x} = 6x^2 = 6 \cdot (2)^2 = 24\\
\frac{\partial f}{\partial y} = 6y = 6 (3) = 18
\end{align}
$$

In [12]:
x = th.tensor(2.0, requires_grad = True)
y = th.tensor(3.0, requires_grad = True)
c = th.tensor(1.0)

f = 2 * x**3 + 3 * y**2 + c

In [13]:
f.backward()

Cuando se invoca a este método, se calculan los gradientes de la función respecto al tensor correspondiente y se acumulan en el tensor `grad` que es una propiedad de todas las instancia de `Tensor` y `Parameter`.

In [14]:
x.grad, y.grad

(tensor(24.), tensor(18.))

In [15]:
print(c.grad)

None


Además, al construir la gráfica de cómputo de una operación, PyTorch almacena la función de retropropagación correspondiente en la propiedad `grad_fn` del tensor resultante. Para el ejemplo anterior, esta sería:

In [16]:
a = x**3
b = 2 * a

d = y**2
e = 3 * d

g = b + e
f = g + c

print(a.grad_fn, b.grad_fn, d.grad_fn, e.grad_fn, g.grad_fn, f.grad_fn)

<PowBackward0 object at 0x7fa55c298b80> <MulBackward0 object at 0x7fa55c298a60> <PowBackward0 object at 0x7fa55c298a90> <MulBackward0 object at 0x7fa55c298ac0> <AddBackward0 object at 0x7fa55c298b20> <AddBackward0 object at 0x7fa55c29a9e0>


Podemos evaluar estas funciones:

In [17]:
a.grad_fn(th.tensor(1.))

tensor(12., grad_fn=<MulBackward0>)

In [18]:
b.grad_fn(th.tensor(1.))

(tensor(2.), None)

In [19]:
a.grad_fn(b.grad_fn(th.tensor(1.))[0])

tensor(24., grad_fn=<MulBackward0>)

In [20]:
print(a.grad_fn(b.grad_fn(g.grad_fn(f.grad_fn(th.tensor(1.))[0])[0])[0]))
print(d.grad_fn(e.grad_fn(g.grad_fn(f.grad_fn(th.tensor(1.))[0])[0])[0]))

tensor(24., grad_fn=<MulBackward0>)
tensor(18., grad_fn=<MulBackward0>)


Cuando se llama al método `backward` se van evaluando las funciones de retropropagación en la gráfica de cómputo desde el último nodo hacia atrás usando la propiedad `next_functions` de `grad_fn` hasta obtener los gradientes de los tensores correspondientes en los nodos hoja. Nota que en algunos casos es necesario mantener los resultados intermedios para poder evaluar esta función y obtener el gradiente correspondiente.

Consideremos ahora la función:

$$
g(\mathbf{m}) =  \sum_{j=1}^d m_j^2
$$

La derivada parcial de esta función respecto a cada elemento de $\mathbf{m}$ estaría dada por

$$
\frac{\partial g(\mathbf{m})}{\partial m_j} = 2\cdot m_j
$$

Definimos esta función, la evalúamos para 100 valores entre -10 y 10 y obtenemos las derivadas parciales respecto a cada uno de los 100 valores (gradiente) usando el método `backward`:

In [21]:
m = th.linspace(start = -10, end = 10, steps = 100, requires_grad = True)
g = (m**2).sum()
g.backward()

print(g.grad_fn, m, m.grad)

<SumBackward0 object at 0x7fa55c298550> tensor([-10.0000,  -9.7980,  -9.5960,  -9.3939,  -9.1919,  -8.9899,  -8.7879,
         -8.5859,  -8.3838,  -8.1818,  -7.9798,  -7.7778,  -7.5758,  -7.3737,
         -7.1717,  -6.9697,  -6.7677,  -6.5657,  -6.3636,  -6.1616,  -5.9596,
         -5.7576,  -5.5556,  -5.3535,  -5.1515,  -4.9495,  -4.7475,  -4.5455,
         -4.3434,  -4.1414,  -3.9394,  -3.7374,  -3.5354,  -3.3333,  -3.1313,
         -2.9293,  -2.7273,  -2.5253,  -2.3232,  -2.1212,  -1.9192,  -1.7172,
         -1.5152,  -1.3131,  -1.1111,  -0.9091,  -0.7071,  -0.5051,  -0.3030,
         -0.1010,   0.1010,   0.3030,   0.5051,   0.7071,   0.9091,   1.1111,
          1.3131,   1.5152,   1.7172,   1.9192,   2.1212,   2.3232,   2.5253,
          2.7273,   2.9293,   3.1313,   3.3333,   3.5354,   3.7374,   3.9394,
          4.1414,   4.3434,   4.5455,   4.7475,   4.9495,   5.1515,   5.3535,
          5.5556,   5.7576,   5.9596,   6.1616,   6.3636,   6.5657,   6.7677,
          6.9697,   7.17

Una vez que se llama al método `.backward`, se elimina el grafo de cómputo. Sin embargo, es posible mantenerlo pasando el argumento `retain_graph=True` en la llamada.

In [22]:
g = (m**2).sum()
g.backward(retain_graph=True)

In [23]:
g.backward()

In [24]:
m.grad

tensor([-60.0000, -58.7879, -57.5758, -56.3636, -55.1515, -53.9394, -52.7273,
        -51.5152, -50.3030, -49.0909, -47.8788, -46.6667, -45.4545, -44.2424,
        -43.0303, -41.8182, -40.6061, -39.3939, -38.1818, -36.9697, -35.7576,
        -34.5455, -33.3333, -32.1212, -30.9091, -29.6970, -28.4848, -27.2727,
        -26.0606, -24.8485, -23.6364, -22.4242, -21.2121, -20.0000, -18.7879,
        -17.5758, -16.3636, -15.1515, -13.9394, -12.7273, -11.5152, -10.3030,
         -9.0909,  -7.8788,  -6.6667,  -5.4545,  -4.2424,  -3.0303,  -1.8182,
         -0.6061,   0.6061,   1.8182,   3.0303,   4.2424,   5.4545,   6.6667,
          7.8788,   9.0909,  10.3030,  11.5152,  12.7273,  13.9394,  15.1515,
         16.3636,  17.5758,  18.7879,  20.0000,  21.2121,  22.4242,  23.6364,
         24.8485,  26.0606,  27.2727,  28.4848,  29.6970,  30.9091,  32.1212,
         33.3333,  34.5455,  35.7576,  36.9697,  38.1818,  39.3939,  40.6061,
         41.8182,  43.0303,  44.2424,  45.4545,  46.6667,  47.87

Múltiples llamadas a la función y al método `backward` acumulan los gradientes en `.grad`. Por lo mismo, en muchas ocasiones es necesarios ponerlos a 0 con el método (_in-pace_) `zero_`.

In [25]:
with th.no_grad():
  m.grad.zero_()
m.grad

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

También podemos diferenciar automáticamente respecto a más de un tensor.

In [26]:
l = th.rand_like(m, requires_grad=True)
h = (m**2 + l**3).sum()
h.backward()
h.grad_fn, m.grad, l.grad

(<SumBackward0 at 0x7fa55c298370>,
 tensor([-20.0000, -19.5960, -19.1919, -18.7879, -18.3838, -17.9798, -17.5758,
         -17.1717, -16.7677, -16.3636, -15.9596, -15.5556, -15.1515, -14.7475,
         -14.3434, -13.9394, -13.5354, -13.1313, -12.7273, -12.3232, -11.9192,
         -11.5152, -11.1111, -10.7071, -10.3030,  -9.8990,  -9.4949,  -9.0909,
          -8.6869,  -8.2828,  -7.8788,  -7.4747,  -7.0707,  -6.6667,  -6.2626,
          -5.8586,  -5.4545,  -5.0505,  -4.6465,  -4.2424,  -3.8384,  -3.4343,
          -3.0303,  -2.6263,  -2.2222,  -1.8182,  -1.4141,  -1.0101,  -0.6061,
          -0.2020,   0.2020,   0.6061,   1.0101,   1.4141,   1.8182,   2.2222,
           2.6263,   3.0303,   3.4343,   3.8384,   4.2424,   4.6465,   5.0505,
           5.4545,   5.8586,   6.2626,   6.6667,   7.0707,   7.4747,   7.8788,
           8.2828,   8.6869,   9.0909,   9.4949,   9.8990,  10.3030,  10.7071,
          11.1111,  11.5152,  11.9192,  12.3232,  12.7273,  13.1313,  13.5354,
          13.9394

Por otro lado, es posible obtener derivadas de orden mayor, pero esto se logra usando la función `grad` del módulo `autograd` en lugar del método `backward`.

In [27]:
z = th.rand(100, requires_grad=True)
f = (z**3).sum()
df = th.autograd.grad(f, z, create_graph=True)[0]
d2f = th.autograd.grad(df.sum(), z)[0]

print(f, f.grad_fn)
print(df, df.grad_fn)
print(z, z.grad)
print(d2f)

tensor(20.5570, grad_fn=<SumBackward0>) <SumBackward0 object at 0x7fa55c26f2e0>
tensor([6.3126e-03, 2.9859e-01, 2.5451e+00, 1.4481e+00, 6.7725e-01, 1.1826e-01,
        1.1302e-01, 8.1484e-03, 3.4075e-01, 1.3421e+00, 2.0114e+00, 1.6024e+00,
        1.0102e-02, 1.1918e-01, 5.3195e-01, 2.9028e+00, 9.8268e-01, 4.1184e-01,
        1.4989e+00, 2.8748e-01, 9.3321e-02, 2.2444e+00, 2.2301e-01, 4.7944e-01,
        2.0247e-05, 2.0898e+00, 2.3170e+00, 1.3963e+00, 6.8732e-02, 1.2792e-04,
        2.6458e-02, 2.2856e+00, 1.6430e+00, 2.5434e+00, 1.7416e+00, 1.1777e+00,
        7.3538e-01, 4.3018e-02, 1.5386e-02, 3.1348e-03, 1.4897e+00, 1.9434e-01,
        4.7850e-01, 1.3515e-01, 5.0157e-01, 6.5785e-02, 9.0091e-02, 1.3301e+00,
        3.7045e-01, 1.9618e+00, 3.4598e-01, 5.3240e-02, 5.0869e-01, 1.9911e-01,
        3.6129e-01, 1.7283e-03, 1.8240e+00, 6.9219e-02, 1.6934e+00, 1.5851e+00,
        2.2045e+00, 4.0699e-02, 2.2167e+00, 2.0849e-01, 1.4099e+00, 2.8201e+00,
        5.5337e-01, 7.3844e-01, 4.4441e-

## Neurona artificial
La salida de una neurona artificial se obtiene multiplicando la transpuesta del vector columna de pesos $\mathbf{w}\in \mathbb{R}^d$ por el vector columna de entrada $\mathbf{x} \in \mathbb{R}^d$, sumando al final el valor del sesgo $b$ y evaluando el resultado con la función de activación $\phi$, esto es

$$
a = \phi\left(\mathbf{w}^\top \mathbf{x} + b\right)
$$

![Diagrama general de la neurona artificial](http://turing.iimas.unam.mx/~gibranfp/cursos/neurona.svg)

Una neurona artificial con función de activación logística o sigmoide entrenada minimizando la [entropía cruzada binaria](https://en.wikipedia.org/wiki/Cross_entropy) se corresponde con una [regresión logística](https://en.wikipedia.org/wiki/Logistic_regression). La [función sigmoide](https://en.wikipedia.org/wiki/Sigmoid_function) o logística está dada por

$$
\sigma(z) = \frac{1}{1 + e^{-z}},
$$

Por lo tanto, la salida de la neurona sigmoide o logística sería

$$
\hat{y} = \sigma(\mathbf{w}^\top \mathbf{x} + b)
$$



In [28]:
def neurona_sigmoide(w, b, X):
  return th.sigmoid(X @ w + b)

Por su parte, la entropía cruzada binaria está dada por
$$
ECB(\mathbf{y}, \mathbf{\hat{y}}) = -\sum_{i=1}^n \left[y^{(i)} \log{(\hat{y}^{(i)})} + (1 - y^{(i)}) \log{(1 - \hat{y}^{(i)}}\right]
$$

In [29]:
def ecb(y, y_hat):
  perdida_unos = th.log(y_hat[y == 1]).sum()
  perdida_ceros = th.log(1 - y_hat[y == 0]).sum()
  return -(perdida_unos + perdida_ceros)

Generamos un conjunto de ejemplos sintéticos (aleatorios).

In [30]:
n = 100
d = 10
X = th.normal(size = (n, d), mean = 0, std = 1)
y = th.randint(low = 0, high = 2, size = (n, 1))

print(X, y)

tensor([[-5.5719e-01, -9.6835e-01,  8.7128e-01, -9.5641e-02,  4.0380e-01,
         -7.1398e-01,  8.3373e-01, -9.5855e-01,  1.0682e+00, -2.5272e-01],
        [-1.8815e-01, -7.7115e-01,  1.7989e-01, -2.1268e+00, -1.3408e-01,
         -1.0408e+00,  7.6942e-01,  2.5574e+00,  5.7161e-01,  1.3596e+00],
        [ 4.3344e-01, -7.1719e-01,  1.0554e+00, -1.4534e+00,  1.7361e+00,
          1.8350e+00,  8.8002e-01,  5.6080e-02,  3.7818e-01,  7.0511e-01],
        [-1.7237e+00, -8.4348e-01, -4.8619e-01, -3.3600e-01,  3.6716e-02,
          4.9340e-01,  8.8538e-01,  1.8244e-01,  7.8638e-01, -5.7920e-02],
        [ 1.3637e-01,  3.0880e-01,  1.6617e+00,  1.7512e-01,  6.0841e-01,
          1.6309e+00, -8.4723e-02,  1.0844e+00,  1.9537e-01, -1.3350e+00],
        [ 3.9451e-01,  1.7060e+00, -7.9394e-01,  3.7523e-01,  8.7910e-02,
         -1.2415e+00, -5.6626e-01,  3.9892e-01,  1.3695e+00, -2.5189e-01],
        [ 1.9003e+00,  1.6951e+00,  2.8090e-02, -1.7537e-01,  4.0854e-01,
         -1.2609e+00,  9.1652e-0

 Creamos tensores para los pesos inicializados con valores aleatorios muestreados de una normal ($\mu = 0$ y $\sigma = 1$) y el sesgo inicializado con cero.

In [31]:
w = nn.parameter.Parameter(th.randn((d, 1)))
b = nn.parameter.Parameter(th.zeros((1, 1)))

print(w, b)

Parameter containing:
tensor([[-1.5897],
        [-0.0405],
        [ 1.9010],
        [-0.6620],
        [ 0.2589],
        [-1.0627],
        [-1.4913],
        [ 0.1673],
        [ 0.7528],
        [ 0.6113]], requires_grad=True) Parameter containing:
tensor([[0.]], requires_grad=True)


Podemos obtener los gradientes de $\mathbf{w}$ y $b$ respecto a la [entropía cruzada binaria](https://en.wikipedia.org/wiki/Cross_entropy) usando diferenciación automática.

![](https://pytorch.org/tutorials/_images/comp-graph.png)

Fuente: Tutorial [Automatic Differentiation with `torch.autograd`](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html).

Nota que en esta gráfica de cómputo no se observa la función sigmoide. Esto se debe a que en PyTorch hay una versión de la entropía cruzada binaria que recibe los _logits_ como entrada.



In [32]:
y_hat = neurona_sigmoide(w, b, X)
fp = ecb(y, y_hat)
fp.backward()

print(w.grad, b.grad)

tensor([[-15.5506],
        [ 10.1311],
        [ 16.7294],
        [ -5.6696],
        [  1.5938],
        [-14.8970],
        [-13.9856],
        [  4.4366],
        [  9.3861],
        [ 12.7488]]) tensor([[-7.1167]])


## Ejercicio
Genera un conjunto de datos sintético, programa la propagación hacia adelante y calcula los gradientes de la función de pérdida de la suma del error cuadrático medio respecto a los pesos y sesgos para $K$ regresiones lineales.

## Entrenando un modelo de regresión con descenso por gradiente
Vamos a entrenar un modelo de regresión lineal con [descenso por gradiente](https://youtu.be/IHZwWFHWa-w) usando la diferenciación automática de Tensorflow.

Primero generamos un conjunto de datos sintético y lo almacenamos en una instancia de `Tensor`.

Definimos las instancias de `Parameter` para $\mathbf{w}$ y $b$ las cuales inicializamos con valores aleatorios (normal con media 0 y desviación estándar 0.1) y con ceros, respectivamente.

Definimos una función para producir un tensor de salidas a partir de un tensor de entradas, esto es,
$$
\hat{y} = b + \mathbf{w}^\top\mathbf{x}.
$$


Ahora definimos nuestro ciclo de entrenamiento en el cual generamos la salida para cada entrada, calculamos los gradientes de $\mathbf{w}$ y $b$ respecto al error cuadrático medio usando la diferenciación automática de Tensorflow y finalmente actualizamos $\mathbf{w}$ y $b$ con la regla de actualización del descenso por gradiente:

$$
\boldsymbol{\theta}^{[t + 1]}   = \boldsymbol{\theta}^{[t]} - \alpha \nabla \mathcal{L}(\boldsymbol{\theta}^{[t]})
$$
donde
$$
\begin{align*}
\boldsymbol{\theta} & = \{\mathbf{w}, \mathbf{b}\}\\
\nabla \mathcal{L}(\boldsymbol{\theta}^{[t]}) & = \left[  \frac{\partial \mathcal{L}}{\partial  \theta_0^{[t]}}, \cdots , \frac{\partial \mathcal{L}}{\partial \theta_d^{[t]}}\right]
\end{align*}
$$    

A $\alpha$ se le conoce como tasa de aprendizaje.


Visualizamos el modelo.

Graficamos el valor de la pérdida por época.

En el entrenamiento de redes neuronales profundas es común usar aproximaciones estocásticas del descenso por gradiente (o variaciones). Estas aproximaciones estiman $\nabla \mathcal{L}(\boldsymbol{\theta}^{[t]})$ y actualizan los parámetros (pesos y sesgos) usando un minilote $\mathcal{B}$ de ejemplos (en lugar de todo el conjunto) de entrenamiento, donde $\vert \mathcal{B} \vert$ es un hiperparámetro. Una estrategia para generar los lotes es dividir y ordenar aleatoriamente el conjunto de $n$ ejemplos de entrenamiento en $k$ minilotes ($\vert \mathcal{B} \vert \times k \approx n$) e ir tomando lotes consecutivos hasta pasar por todo el conjunto. Aquí una época ocurre cada vez que se han considerado los $k$ minilotes.

Visualizamos el modelo.

Graficamos el valor de la pérdida por época.